In [1]:
import numpy as np
from scipy.constants import hbar,h,e,c
from scipy.constants import elementary_charge as elc
F0 = 2.0678e-15

In [2]:
e_eff=(11.45+1)/2## for all calculations
Mhz=(2*np.pi*1e6)## divide on it to obtain value in mhz
j=0.5e-3      # critical current density nA/nm^2

In [3]:
import pandas as pd

# Q1 related parameters

In [4]:
def cap_func(Q1,Qg1,Qg2):
    return Q1+(1/Qg1+1/Qg2)**(-1)

In [5]:
def Ec(C):#fF
    return np.round((elc)**2/(2*C)/h/1e9*1e15,1) # return in MHz

In [6]:
# path=r'E:\Github2.0\QCreator\designs\ThreeQubitsPlusTwoQubits\maxwell_simulations'
path=r'C:\Python\Github\QCreator\designs\ThreeQubitsPlusTwoQubits'
# path=r'D:\NextCloud\Quantum Computing\ChipDesigns\Ivan_Gerhard_designs\Two_qubits_plus_coupler\Hamiltonian_creation'

In [7]:
data=pd.read_csv(path+'\Design_single_qubit_coupler.txt',sep='\t')# matrix should be done on pF

In [8]:
data

,Charge,ground,Q1_bottom,Q1_top,Resonator
Charge,2.662900e-03,-0.002635,-0.000001,-0.000026,-1.177400e-07
ground,-2.635300e-03,0.720550,-0.080903,-0.553330,-8.367800e-02
Q1_bottom,-1.294600e-06,-0.080903,0.127730,-0.039377,-7.446900e-03
Q1_top,-2.611200e-05,-0.553330,-0.039377,0.593680,-9.405700e-04
Resonator,-1.177400e-07,-0.083678,-0.007447,-0.000941,9.206500e-02


In [9]:
q_name='Q1_'
Q1_cap_ground_top=data[q_name+'top'][3]+data[q_name+'top'][2]
Q1_cap_ground_bottom=data[q_name+'bottom'][2]+data[q_name+'bottom'][3]
Q1_cap_top_bottom=-data[q_name+'bottom'][3]
Q1_cap=cap_func(Q1_cap_top_bottom,Q1_cap_ground_top,Q1_cap_ground_bottom)

In [10]:
Q1_cap_top_bottom,Q1_cap_ground_bottom,Q1_cap_ground_top

(0.039377, 0.08835300000000001, 0.554303)

In [11]:
Q1_cap

0.11558313976839865

In [12]:
Ec(Q1_cap)

167.6

In [13]:
def I(f,Ec):
    Ej=Ejf(f,Ec)
    I=Ej*2*np.pi/F0*h*1e18
    return I
def Ejf(f,Ec):
    Ej=(f+Ec)**2/(8*Ec)
    return Ej
def Eji(I):#nA
    return I*F0/(2*np.pi)/1e9/h/1e9
def IEj(Ej):
    return Ej/(F0/(Q2*np.pi)/1e9/h/1e9)
def f(Ej,Ec):
    return np.sqrt(8*Ej*Ec)-Ec
def Ec(C):#fF
    return elc**2/(2*C)/h/1e9*1e15

In [14]:
a1    = 0.300
a2    = 0.300 # Junction width in um
#Calculate Qubit frequency:
JJ_area = a1*a2*1e6 #in nm^2
print('Critical current density is: ',j*1e3, 'uA/um^2')
Ic = JJ_area*j #in nA
print('Critical current: ',Ic ,' nA')
print('Ej: ',Eji(Ic) ,' GHz')

print('junction area  in um: ',JJ_area*1e-6)

Critical current density is:  0.5 uA/um^2
Critical current:  45.0  nA
Ej:  22.35039212442379  GHz
junction area  in um:  0.09


In [15]:
f(Eji(Ic),Ec(Q1_cap)/1e3)

5.306449714424038

In [16]:
Eji(Ic)/(Ec(Q1_cap)/1e3)

133.36592218384865

### Resonator coupling

In [17]:
def line_qubit_coupling(Q1g,Q2g,Q1gc,Q2gc):
    return np.abs((Q1g*Q2gc-Q2g*Q1gc)/(Q1g+Q2gc+Q2g+Q1gc))

In [18]:
def cap_pretty_print(args,coeff):
    for i in args:
        print(np.round(i*coeff,1))# return in fF

In [19]:
Q1_cap_res_top=-data[q_name+'top'][-1]
Q1_cap_res_bottom=-data[q_name+'bottom'][-1]
Ccr=line_qubit_coupling(Q1_cap_ground_top,Q1_cap_ground_bottom,Q1_cap_res_top,Q1_cap_res_bottom)

In [20]:
cap_pretty_print([Q1_cap_res_top,Q1_cap_res_bottom,Ccr],1e3)

0.9
7.4
6.2


In [21]:
def qubit_res_coupling(Cc,Cq,wq,wr,Zr):# capacitances in fF, frequencies in GHz
    g= Cc*10**(-12)/2*np.sqrt((2*np.pi*wq*10**(9)*Zr/(Cq*10**(-12))))*wr*10**(9)
    return np.round(g/1e6,1)# return g/2pi in MHz

In [45]:
Zr=52.6
omega_r=6
omega_q=7.5
gqr=qubit_res_coupling(Ccr,Q1_cap,omega_q,omega_r,Zr)
gqr

86.3

### Dispersive shift

In [46]:
def dispersive_shift(gqr,wq,wr,Ec): # gqr in MHz,wq,wr in GHZ, EC in MHz and positive
    chi=gqr**2/((wq-wr)*10**(3))/(1+(wq-wr)*10**(3)/(-Ec))
    return chi # in MHz

In [47]:
chi=dispersive_shift(gqr,omega_q,omega_r,Ec(Q1_cap))
chi

-0.6244989883055101

### Readout resonator properties

In [48]:
Q=10000
k=2*np.pi*omega_r*1e9/Q 
np.round(1/k*1e9,1), np.round(k/Mhz,2)# first value in ns

(265.3, 0.6)

In [49]:
Tp=(1/k)*((omega_q-omega_r)*10**3/gqr)**2
Tp*1e6#us

80.13639619729172

In [50]:
fmin=4.5
fmax=7.5

In [51]:
Ejf(fmin,Ec(Q1_cap)/1e3),Ejf(fmax,Ec(Q1_cap)/1e3)

(16.250045367683057, 43.851773359135066)

In [52]:
(I(fmin,Ec(Q1_cap)/1e3)+I(fmax,Ec(Q1_cap)/1e3))/2,(I(fmax,Ec(Q1_cap)/1e3)-I(fmin,Ec(Q1_cap)/1e3))/2

(60.50412510998713, 27.786486981989857)

In [53]:
(I(fmin,Ec(Q1_cap)/1e3)+I(fmax,Ec(Q1_cap)/1e3))/2/j/250,(I(fmax,Ec(Q1_cap)/1e3)-I(fmin,Ec(Q1_cap)/1e3))/2/j/100

(484.03300087989703, 555.7297396397971)

In [56]:
np.sqrt(250*484),np.sqrt(556*100)

(347.85054261852173, 235.79652245103193)

In [55]:
Ejf(fmin,Ec(Q1_cap)/1e3)/(Ec(Q1_cap)/1e3),Ejf(fmax,Ec(Q1_cap)/1e3)/(Ec(Q1_cap)/1e3)

(96.96484401372895, 261.6657533738437)